# Workplace for: Customer: 18; DFU: Рис басмати 500 гр

In [6]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from lightgbm import LGBMRegressor

In [7]:
df = pd.read_csv("data_grouped/train_group_18_Рис басмати 500 гр.csv")
sales_2_df_t = pd.read_csv("data_test_grouped/test_group_18_Рис басмати 500 гр.csv")
df, sales_2_df_t

(                    DFU  Customer      Period  BPV  Total Sell-in  Season  \
 0    Рис басмати 500 гр        18  2022-01-24  NaN            NaN  Winter   
 1    Рис басмати 500 гр        18  2022-01-31  NaN            NaN  Winter   
 2    Рис басмати 500 гр        18  2022-02-07  NaN            NaN  Winter   
 3    Рис басмати 500 гр        18  2022-02-14  NaN            NaN  Winter   
 4    Рис басмати 500 гр        18  2022-02-21  NaN            NaN  Winter   
 ..                  ...       ...         ...  ...            ...     ...   
 240  Рис басмати 500 гр        18  2021-12-20  NaN            NaN  Winter   
 241  Рис басмати 500 гр        18  2021-12-27  NaN            NaN  Winter   
 242  Рис басмати 500 гр        18  2022-01-03  NaN            NaN  Winter   
 243  Рис басмати 500 гр        18  2022-01-10  NaN            NaN  Winter   
 244  Рис басмати 500 гр        18  2022-01-17  NaN            NaN  Winter   
 
             Type Geography End of Period  BPV_sale_period  So

## Visualization

In [8]:
unique_stat_colors = {
    'BPV Mean': '#1f77b4',
    'BPV Median': '#ff7f0e',
    'BPV CI Low (95%)': '#2ca02c',
    'BPV CI Up (95%)': '#d62728',
    'Total Sell-in Mean': '#9467bd',
    'Total Sell-in Median': '#8c564b',
    'Total Sell-in CI Low (95%)': '#e377c2',
    'Total Sell-in CI Up (95%)': '#7f7f7f',
}

for (dfu, customer), group in df.groupby(['DFU', 'Customer']):
    melted_group = group.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                              var_name='Metric', value_name='Value')

    stat_lines = []

    for metric in ['BPV', 'Total Sell-in']:
        data = group[metric].dropna()
        if not data.empty:
            mean_val = data.mean()
            median_val = data.median()
            std_val = data.std()

            stat_lines.extend([
                (metric, f'{metric} Mean: {mean_val:.2f}', mean_val, unique_stat_colors[f'{metric} Mean']),
                (metric, f'{metric} Median: {median_val:.2f}', median_val, unique_stat_colors[f'{metric} Median'])
            ])

    fig = px.line(
        melted_group,
        x='Period', y='Value', color='Metric',
        title=f'DFU: {dfu} | Customer: {customer}',
        labels={'Value': 'Sales', 'Period': 'Date'}
    )

    for metric, label, value, color in stat_lines:
        fig.add_trace(go.Scatter(
            x=[group['Period'].min(), group['Period'].max()],
            y=[value, value],
            mode='lines',
            name=label,
            line=dict(dash='dot', color=color, width=1.5),
            showlegend=True
        ))

    for metric in ['BPV', 'Total Sell-in']:
        metric_df = group[['Period', metric]].dropna()
        if not metric_df.empty:
            metric_df = metric_df.copy()
            metric_df['Period_ordinal'] = pd.to_datetime(metric_df['Period']).map(pd.Timestamp.toordinal)
            X = metric_df[['Period_ordinal']]
            y = metric_df[metric]
            model = LinearRegression().fit(X, y)
            y_pred = model.predict(X)

            fig.add_trace(go.Scatter(
                x=metric_df['Period'],
                y=y_pred,
                mode='lines',
                name=f'Trend - {metric}',
                line=dict(dash='dash', color='orange')
            ))

    if group[['BPV', 'Total Sell-in']].dropna().shape[0] > 1:
        corr = group['BPV'].corr(group['Total Sell-in'])
        fig.add_annotation(
            text=f"Корреляция BPV и Total Sell-in: {corr:.2f}",
            xref="paper", yref="paper",
            x=0.99, y=1.05, showarrow=False,
            font=dict(size=12, color="white"),
            align="right",
            bordercolor="white", borderwidth=1
        )

    filtered_sales_2 = sales_2_df_t[(sales_2_df_t['DFU'] == dfu) & (sales_2_df_t['Customer'] == customer)]
    if not filtered_sales_2.empty:
        melted_sales_2 = filtered_sales_2.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                                               var_name='Metric', value_name='Value')
        for metric in melted_sales_2['Metric'].unique():
            metric_data = melted_sales_2[melted_sales_2['Metric'] == metric]
            color = 'black' if metric == 'BPV' else 'gray'
            fig.add_trace(go.Scatter(
                x=metric_data['Period'],
                y=metric_data['Value'],
                mode='lines+markers',
                name=f'test_sales - {metric}',
                line=dict(color=color, width=2, dash='dot')
            ))

    fig.update_xaxes(title_text='Дата', tickformat='%Y-%m-%d')
    fig.update_yaxes(title_text='Значение')
    fig.show()


## Machine learning

In [9]:
def compute_metrics(y_true, y_pred, m=1):
    """
    Compute forecast accuracy metrics: WAPE, MAE, RMSE, MASE.

    Parameters
    ----------
    y_true : array-like of shape (n,)
        Истинные значения.
    y_pred : array-like of shape (n,)
        Предсказанные значения.
    m : int, default=1
        Шаг «наивного» прогноза для расчёта MASE.
        Для несезонного ряда обычно m=1;
        для сезонного — период сезонности.

    Returns
    -------
    dict
        {
            'WAPE': float,  # в процентах
            'MAE': float,
            'RMSE': float,
            'MASE': float
        }
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    # WAPE
    num = np.abs(y_true - y_pred).sum()
    denom = np.abs(y_true).sum()
    wape = num / denom * 100 if denom != 0 else np.nan

    # MAE
    mae = np.mean(np.abs(y_true - y_pred))

    # RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    # MASE
    if n > m:
        naive_errors = np.abs(y_true[m:] - y_true[:-m])
        scale = np.mean(naive_errors)
        mase = mae / scale if scale != 0 else np.nan
    else:
        mase = np.nan

    return {
        'WAPE': wape,
        'MAE': mae,
        'RMSE': rmse,
        'MASE': mase
    }

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor

# Пути к данным
train_path = "data_grouped/train_group_18_Рис басмати 500 гр.csv"
test_path = "data_test_grouped/test_group_18_Рис басмати 500 гр.csv"

# Загрузка
train = pd.read_csv(train_path)
train = train[(train["Period"] >= "2021-02-05") & (train["Period"] <= "2022-01-17")].copy()
test = pd.read_csv(test_path).rename(columns={"BPV": "BPV_true"})

# Обогащение
def enrich(df):
    df["Period"] = pd.to_datetime(df["Period"])
    df["Period_ord"] = df["Period"].map(pd.Timestamp.toordinal)
    df["weekofyear"] = df["Period"].dt.isocalendar().week.astype(int)
    df["month"] = df["Period"].dt.month
    df["day"] = df["Period"].dt.day
    df["days_in_month"] = df["Period"].dt.days_in_month
    df["week_share"] = df["day"] / df["days_in_month"]
    df["sin_month"] = np.sin(2 * np.pi * df["month"] / 12)
    df["cos_month"] = np.cos(2 * np.pi * df["month"] / 12)
    df["Period_last_year"] = df["Period"] - pd.DateOffset(weeks=52)
    return df

train = enrich(train)
test = enrich(test)

# Sell-in год назад
sellin_map = train[["Period", "Total Sell-in"]].rename(
    columns={"Period": "Period_last_year", "Total Sell-in": "sellin_last_year"}
)
train = train.merge(sellin_map, on="Period_last_year", how="left")
test = test.merge(sellin_map, on="Period_last_year", how="left")
train["sellin_last_year"].fillna(0, inplace=True)
test["sellin_last_year"].fillna(0, inplace=True)

train["log_sellin_last_year"] = np.log1p(train["sellin_last_year"])
test["log_sellin_last_year"] = np.log1p(test["sellin_last_year"])

# Фичи
feat_cols = [
    "Period_ord", "weekofyear", "month",
    "sellin_last_year", "log_sellin_last_year",
    "week_share", "sin_month", "cos_month"
]

# Подготовка train
train_clf = train[train["BPV"].notna()].copy()
train_clf["sale_flag"] = (train_clf["BPV"] > 0).astype(int)

X_clf = train_clf[feat_cols]
y_clf = train_clf["sale_flag"]

train_reg = train_clf[train_clf["sale_flag"] == 1].copy()
X_reg = train_reg[feat_cols]
y_reg = np.log1p(train_reg["BPV"])

# Тест
X_test = test[feat_cols].reset_index(drop=True)
true_test = test[["Period", "BPV_true"]].reset_index(drop=True)

# Ансамбль XGBoost
N_MODELS = 5
probas = []
preds = []

for seed in range(42, 42 + N_MODELS):
    clf = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        scale_pos_weight=(y_clf == 0).sum() / (y_clf == 1).sum(),
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=seed,
        verbosity=0
    )
    clf.fit(X_clf, y_clf)
    proba_test = clf.predict_proba(X_test)[:, 1]
    probas.append(proba_test)

    reg = XGBRegressor(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=5,
        random_state=seed,
        verbosity=0
    )
    reg.fit(X_reg, y_reg)
    pred = np.expm1(reg.predict(X_test))
    preds.append(np.maximum(pred, 0))

# Медианные предсказания
proba_test_median = np.median(probas, axis=0)
base_pred_median = np.median(preds, axis=0)

# Порог
threshold = 0.1
is_sale = proba_test_median >= threshold
y_pred = np.where(is_sale, base_pred_median, 0)

# Объединение и метрики
merged = pd.DataFrame({
    "Period": true_test["Period"],
    "BPV_true": true_test["BPV_true"],
    "BPV_pred": y_pred
})

metrics = compute_metrics(merged["BPV_true"], merged["BPV_pred"], m=1)

print("\n📊 Метрики медианного ансамбля XGBoost:")
for k, v in metrics.items():
    print(f"{k} = {v:.4f}")

/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_3690/34998010.py:38: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_3690/34998010.py:39: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[c


📊 Метрики медианного ансамбля XGBoost:
WAPE = 75.0000
MAE = 1.1286
RMSE = 1.1896
MASE = nan


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_true"],
    mode="markers+lines",
    name="BPV_true",
    line=dict(color="blue"),
    marker=dict(size=6)
))
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_pred"],
    mode="markers+lines",
    name="BPV_pred",
    line=dict(color="red", dash="dot"),
    marker=dict(size=6)
))
fig.update_layout(
    title="Сравнение BPV_true и BPV_pred (с обновленным набором признаков)",
    xaxis_title="Period",
    yaxis_title="BPV",
    legend_title="Легенда"
)
fig.show()

## Revision

1. Оформил ноутбук группы (Макух Д. В.)